In [11]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn import metrics
from sklearn.decomposition import PCA
import math
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from keras.layers import Dense
from gurobipy import Model, GRB, quicksum

In [12]:
warnings.filterwarnings('ignore')

In [13]:
# Load the dataframes
%store -r df_yahoo

In [14]:
def predicted_zscores():
    
    df_predicted = pd.read_csv('Forecasted_Player_Stats.csv')

    # Selecting columns
    df_predicted_zscores = df_predicted[["Player", "Forecasted G", "Forecasted PTS", "Forecasted TRB", "Forecasted AST", "Forecasted 3P", "Forecasted STL"]] 
    df_predicted_zscores["Average Games Played"] = df_predicted_zscores["Forecasted G"]/82
    
    # Calculate the means
    pts_mean = np.average(df_predicted_zscores["Forecasted PTS"].dropna())
    reb_mean = np.average(df_predicted_zscores["Forecasted TRB"].dropna())
    ast_mean = np.average(df_predicted_zscores["Forecasted AST"].dropna())
    tp_mean = np.average(df_predicted_zscores["Forecasted 3P"].dropna())
    stl_mean = np.average(df_predicted_zscores["Forecasted STL"].dropna())

    # Calculate the stds
    pts_std = np.std(df_predicted_zscores["Forecasted PTS"].dropna())
    reb_std = np.std(df_predicted_zscores["Forecasted TRB"].dropna())
    ast_std = np.std(df_predicted_zscores["Forecasted AST"].dropna())
    tp_std = np.std(df_predicted_zscores["Forecasted 3P"].dropna())
    stl_std = np.std(df_predicted_zscores["Forecasted STL"].dropna())

    # Calculate the z-scores
    df_predicted_zscores["PTS Z-Score"] = (df_predicted_zscores["Forecasted PTS"] - pts_mean) / pts_std
    df_predicted_zscores["TRB Z-Score"] = (df_predicted_zscores["Forecasted TRB"] - reb_mean) / reb_std
    df_predicted_zscores["AST Z-Score"] = (df_predicted_zscores["Forecasted AST"] - ast_mean) / ast_std
    df_predicted_zscores["3P Z-Score"] = (df_predicted_zscores["Forecasted 3P"] - tp_mean) / tp_std
    df_predicted_zscores["STL Z-Score"] = (df_predicted_zscores["Forecasted STL"] - stl_mean) / stl_std
    
    return df_predicted_zscores

In [15]:
df_predicted_zscores = predicted_zscores()

In [16]:
def MLR():
    
    # MLR method
    df_matchup = pd.read_csv('Yahoo_Matchup_List.csv')
    df_matchup = df_matchup.drop(["Team 1 ID", "Team 2 ID", "Playoff", "Consolation", "Complete", "Team 1 Name", "Team 2 Name", "Week", "Team 1 TO", "Team 2 TO", "Team 1 FG%", "Team 2 FG%", "Team 1 FT%", "Team 2 FT%", "Team 1 BLK", "Team 2 BLK"], axis=1)

    df_copy_1 = df_matchup[["Team 1 Points", "Team 1 3PTM", "Team 1 PTS", "Team 1 REB", "Team 1 AST", "Team 1 ST"]]
    df_copy_2 = df_matchup[["Team 2 Points", "Team 2 3PTM", "Team 2 PTS", "Team 2 REB", "Team 2 AST", "Team 2 ST"]]

    df_copy_1 = df_copy_1.rename(columns={"Team 1 Points": "Team Points", "Team 1 3PTM": "Team 3PTM", "Team 1 PTS": "Team PTS", "Team 1 REB": "Team REB", "Team 1 AST": "Team AST", "Team 1 ST": "Team ST"})
    df_copy_2 = df_copy_2.rename(columns={"Team 2 Points": "Team Points", "Team 2 3PTM": "Team 3PTM", "Team 2 PTS": "Team PTS", "Team 2 REB": "Team REB", "Team 2 AST": "Team AST", "Team 2 ST": "Team ST"})
    
    df_matchup = pd.concat([df_copy_1, df_copy_2], axis=0, ignore_index=True)
    
    print(df_matchup)
    
    y = df_matchup["Team Points"]
    X = df_matchup[["Team 3PTM", "Team PTS", "Team REB", "Team AST", "Team ST"]]
    
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    
    x = sm.add_constant(X)

    model = sm.OLS(y, x).fit()
    predictions = model.predict(x) 

    print_model = model.summary()
     
    MAE = metrics.mean_absolute_error(df_matchup["Team Points"], predictions)
    MSE = metrics.mean_squared_error(df_matchup["Team Points"], predictions)
    R2 = np.sqrt(metrics.mean_squared_error(df_matchup["Team Points"], predictions))
    
    print('Mean Absolute Error:', MAE)
    print('Mean Square Error:', MSE)
    print('Root Mean Square Error:', R2)
    print("\n")
    
    return

In [17]:
MLR()

     Team Points  Team 3PTM  Team PTS  Team REB  Team AST  Team ST
0              5         64       563       212       116       42
1              6         46       485       195       136       38
2              7         46       433       221        83       27
3              4         63       654       217       123       35
4              4         52       496       199       105       21
..           ...        ...       ...       ...       ...      ...
247            4         58       525       209       101       27
248            3         70       710       248       200       53
249            4         80       731       297       218       44
250            3         49       538       216       121       22
251            4         61       517       214       102       33

[252 rows x 6 columns]
Intercept: 
 1.0646977157606567
Coefficients: 
 [ 0.01215019  0.00152018  0.00257365 -0.00033497  0.02304029]
Mean Absolute Error: 1.3444389933657834
Mean Square Error: 2.6

In [18]:
def pca_weights(df_predicted_zscores):
    
    # PCA
    df_pca = df_predicted_zscores[["Forecasted PTS", "Forecasted TRB", "Forecasted AST", "Forecasted 3P", "Forecasted STL"]]
    
    X = df_pca.values
    
    # Perform PCA
    pca = PCA(n_components=1)
    pca.fit(X)

    # Weights
    weights = pca.components_[0]
    print("PCA Weights:", weights)
    
    return

In [19]:
pca_weights(df_predicted_zscores)

PCA Weights: [0.96461031 0.10425928 0.23006069 0.07510367 0.00940629]


In [25]:
def writing_to_csv(df_predicted_zscores):
    
    # PCA
    df_predicted_zscores["Total PCA"] = (0.96461031 * df_predicted_zscores["Forecasted PTS"] + 0.10425928 * df_predicted_zscores["Forecasted TRB"] + 0.23006069 * df_predicted_zscores["Forecasted AST"] + 0.07510367 * df_predicted_zscores["Forecasted 3P"] + 0.00940629 * df_predicted_zscores["Forecasted STL"])*df_predicted_zscores["Average Games Played"]
    
    df_predicted_zscores = df_predicted_zscores.sort_values(by='Total PCA', ascending=False, ignore_index=True)
    
    df_predicted_zscores.to_csv('Predicted_PCA.csv', index=False)
    
    # MLR
    df_predicted_zscores["Total MLR"] = (0.00152018 * df_predicted_zscores["Forecasted PTS"] + 0.00257365 * df_predicted_zscores["Forecasted TRB"] + -0.00033497 * df_predicted_zscores["Forecasted AST"] + 0.01215019 * df_predicted_zscores["Forecasted 3P"] + 0.02304029 * df_predicted_zscores["Forecasted STL"])*df_predicted_zscores["Average Games Played"]

    df_predicted_zscores = df_predicted_zscores.sort_values(by='Total MLR', ascending=False, ignore_index=True)
    
    df_predicted_zscores.to_csv('Predicted_MLR.csv', index=False) 
    
    # Yahoo Analysis
    df_predicted_zscores["Total Yahoo"] = (1.38 * df_predicted_zscores["Forecasted PTS"] + 1.3 * df_predicted_zscores["Forecasted TRB"] + 1.41 * df_predicted_zscores["Forecasted AST"] + 1.23 * df_predicted_zscores["Forecasted 3P"] + 1.16 * df_predicted_zscores["Forecasted STL"])*df_predicted_zscores["Average Games Played"]

    df_predicted_zscores = df_predicted_zscores.sort_values(by='Total Yahoo', ascending=False, ignore_index=True)
    
    df_predicted_zscores.to_csv('Predicted_Yahoo.csv', index=False) 
    
    # NBA Analysis
    df_predicted_zscores["Total NBA"] = (1 * df_predicted_zscores["Forecasted PTS"] + 1.5 * df_predicted_zscores["Forecasted TRB"] + 1.5 * df_predicted_zscores["Forecasted AST"] + 1.25 * df_predicted_zscores["Forecasted 3P"] + 1.25 * df_predicted_zscores["Forecasted STL"])*df_predicted_zscores["Average Games Played"]

    df_predicted_zscores = df_predicted_zscores.sort_values(by='Total NBA', ascending=False, ignore_index=True)
    
    df_predicted_zscores.to_csv('Predicted_NBA.csv', index=False) 
    
    return

In [26]:
writing_to_csv(df_predicted_zscores)

In [30]:
# Helper method
def optimize_roster(df_yahoo, filename, column):
    
    df_optimal = pd.read_csv(filename)
    df_optimal = df_optimal[["Player", column]]
    df_optimal = pd.merge(df_optimal, df_yahoo, on=['Player'], how="left")
    df_optimal["Round"] = 1
    df_optimal = df_optimal[:156]

    # Rounds correlating to players
    df_optimal["Round"][0:12] = 1
    df_optimal["Round"][12:24] = 2
    df_optimal["Round"][24:36] = 3
    df_optimal["Round"][36:48] = 4
    df_optimal["Round"][48:60] = 5
    df_optimal["Round"][60:72] = 6
    df_optimal["Round"][72:84] = 7
    df_optimal["Round"][84:96] = 8
    df_optimal["Round"][96:108] = 9
    df_optimal["Round"][108:120] = 10
    df_optimal["Round"][120:132] = 11
    df_optimal["Round"][132:144] = 12
    df_optimal["Round"][144:156] = 13

    # Convert column into lists
    df_optimal['Eligible Positions'] = df_optimal['Eligible Positions'].apply(lambda x: x.split(','))

    # Convert dataframe into dictionary
    result_dict = df_optimal.set_index('Player').T.to_dict('list')

    # Formatting
    final_dict = {player: [pca, positions, round] for player, (pca, positions, round) in result_dict.items()}

    # Optimization Model
    m = Model("Roster Optimization")

    # Variables
    x = {}
    for i in range(156):  # Adjusted to 156 players
        x[i] = m.addVar(vtype=GRB.BINARY, name=f'x_{i}')

    # Objective function
    obj = quicksum(x[i] * df_optimal[column].iloc[i] for i in range(156))
    m.setObjective(obj, GRB.MAXIMIZE)

    # Constraints
    m.addConstr(quicksum(x[i] for i in range(156)) == 13, name="Select_13_Players")

    def get_position_constraint(position):
        return quicksum(x[i] for i in range(156) if position in final_dict[df_optimal['Player'].iloc[i]][1])

    m.addConstr(get_position_constraint('PG') >= 1, name="Select_1_PG")
    m.addConstr(get_position_constraint('SG') >= 1, name="Select_1_SG")
    m.addConstr(get_position_constraint('PG') + get_position_constraint('SG') >= 1, name="Select_1_PG_or_SG")
    m.addConstr(get_position_constraint('SF') >= 1, name="Select_1_SF")
    m.addConstr(get_position_constraint('PF') >= 1, name="Select_1_PF")
    m.addConstr(get_position_constraint('SF') + get_position_constraint('PF') >= 1, name="Select_1_SF_or_PF")
    m.addConstr(get_position_constraint('C') >= 1, name="Select_1_C")

    rounds = df_optimal["Round"].unique()
    for round_num in rounds:
        m.addConstr(quicksum(x[i] for i in range(156) if final_dict[df_optimal['Player'].iloc[i]][2] == round_num) <= 1,
                    name=f"Limit_Round_{round_num}")

    m.optimize()

    # Results
    chosen_players = []
    for i in range(156):
        if x[i].x > 0.5:  # If the player is selected (binary variable is 1)
            chosen_players.append(df_optimal['Player'].iloc[i])

    print("Chosen Players:")
    for player in chosen_players:
        print(player)
        print(final_dict[player])
    
    return

In [31]:
def optimal_roster(df_yahoo):
    
    optimize_roster(df_yahoo, 'Predicted_PCA.csv', 'Total PCA')
    optimize_roster(df_yahoo, 'Predicted_MLR.csv', 'Total MLR')
    optimize_roster(df_yahoo, 'Predicted_Yahoo.csv', 'Total Yahoo')
    optimize_roster(df_yahoo, 'Predicted_NBA.csv', 'Total NBA')
    
    return

In [32]:
optimal_roster(df_yahoo)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 156 columns and 787 nonzeros
Model fingerprint: 0x1056c658
Variable types: 0 continuous, 156 integer (156 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [9e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 214.4491799
Presolve removed 2 rows and 96 columns
Presolve time: 0.00s
Presolved: 19 rows, 60 columns, 240 nonzeros
Found heuristic solution: objective 235.5669510
Variable types: 0 continuous, 60 integer (60 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 235.567 214.449 

Optimal solution found (tolerance 1.00e-04)
Best ob